In [1]:
#----- data handling -----#
import pandas as pd
import numpy as np
from PIL import Image

In [2]:
#----- clustering -----#
from sklearn.cluster import KMeans

### Functions definition

In [3]:
def draw_image(array, R, C):
    array = array.reshape((R, C));
    img = Image.fromarray(255 - array, 'P');
    img.show();

In [28]:
def decode(S: bytes):
    return int.from_bytes(S, byteorder='big');

def parse_images(k: int):
    #----- read a file -----#
    f = open("./train-images-idx3-ubyte", "rb");
    assert(decode(f.read(4)) == 2051);
    N = decode(f.read(4));
    assert(k <= N);
    #----- parse image data -----#
    R = decode(f.read(4));
    C = decode(f.read(4));
    assert(R == 28 and C == 28);
    images = [];
    for j in range(k):
        bits = [decode(f.read(1)) for i in range(R*C)];
        img = np.array(bits, dtype=np.uint8)
        images.append(img);
    f.close();
    return images, R, C;

def parse_labels(k: int):
    #----- read a file -----#
    f = open("./train-labels-idx1-ubyte", "rb");
    assert(decode(f.read(4)) == 2049);
    N = decode(f.read(4));
    assert(k <= N);
    #----- parse label data -----#
    result = [decode(f.read(1)) for i in range(k)]
    f.close();
    return result;

def parse_data(k: int):
    #----- create a dataset -----#
    imgs, R, C = parse_images(k);
    labs = parse_labels(k);
    return pd.DataFrame({'image': imgs, 'label': labs}), R, C;

### In this code a clustering algorithm K-means is appied to recognize digits from MNIST dataset. The data is initially marked up and all image data is split into 10 classes, each representing a single digit. 
### We apply ``K-means`` with 10 classes and ``k-means++`` for cluster initialization. After that we compare resulting clusters with benchmark classes.

In [29]:
N = 10000;
df, R, C = parse_data(N);

### Launch K-means

In [23]:
X = df.iloc[:, 0].values;
X = np.stack(X);

kmean_res = KMeans(
    n_clusters=10, 
    init='k-means++'
).fit(X);
kmean_labs = kmean_res.labels_;

real_labs = df.iloc[:, 1].values;

### To compare the resulting clusters with the benchmark, a "similarity" matrix is computed. It is a matrix of 10 rows and 10 columns, containing numbers ``0 <= x <= 1``. Each row represents a benchmark class (i-th row -- digit i) and each column represents one of the obtained clusters. Each matrix cell (i, j) represents a measure of similarity of i-th bechmark class and j-th obtained cluster.

In [26]:
assert(len(kmean_labs) == len(real_labs));
kmean_batch = [np.argwhere(kmean_labs == i).flatten() for i in range(10)];
real_batch  = [np.argwhere(real_labs  == i).flatten() for i in range(10)];
Kset = [set(b) for b in kmean_batch];
Rset = [set(b) for b in real_batch];

metric = [
    [len(Rset[i].intersection(Kset[j])) / 
     len(Kset[j])
    for j in range(10)] 
for i in range(10)];

metric = pd.DataFrame(metric, columns=[c for c in 'ABCDEFGHIJ']);
metric = metric.round(3);

print(metric)

       A      B      C      D      E      F      G      H      I      J
0  0.097  0.001  0.000  0.037  0.002  0.946  0.008  0.045  0.019  0.002
1  0.002  0.001  0.768  0.001  0.001  0.000  0.003  0.000  0.001  0.411
2  0.024  0.002  0.088  0.051  0.007  0.007  0.905  0.027  0.024  0.087
3  0.212  0.015  0.005  0.513  0.013  0.007  0.040  0.011  0.014  0.071
4  0.000  0.225  0.033  0.000  0.239  0.000  0.007  0.016  0.399  0.034
5  0.221  0.050  0.004  0.198  0.014  0.011  0.003  0.027  0.031  0.175
6  0.010  0.000  0.007  0.004  0.000  0.015  0.011  0.852  0.104  0.091
7  0.001  0.355  0.049  0.000  0.427  0.001  0.007  0.000  0.117  0.038
8  0.433  0.029  0.037  0.183  0.024  0.005  0.016  0.018  0.028  0.068
9  0.001  0.322  0.008  0.014  0.273  0.007  0.000  0.003  0.264  0.022


### Filter nonmaximal row elements

In [27]:
maximizers = metric;
for i in range(10):
    m = max(maximizers.iloc[i, :]);
    for j in range(10):
        if maximizers.iloc[i, j] != m:
            maximizers.iloc[i, j] = 0;
print(maximizers)

       A      B      C      D      E      F      G      H      I    J
0  0.000  0.000  0.000  0.000  0.000  0.946  0.000  0.000  0.000  0.0
1  0.000  0.000  0.768  0.000  0.000  0.000  0.000  0.000  0.000  0.0
2  0.000  0.000  0.000  0.000  0.000  0.000  0.905  0.000  0.000  0.0
3  0.000  0.000  0.000  0.513  0.000  0.000  0.000  0.000  0.000  0.0
4  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.399  0.0
5  0.221  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.0
6  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.852  0.000  0.0
7  0.000  0.000  0.000  0.000  0.427  0.000  0.000  0.000  0.000  0.0
8  0.433  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.0
9  0.000  0.322  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.0


### Ideally we would have obtained a matrix which can be diagonalized by applying columns permutations. And the diagolal elenents of the new matrix would have been close to 1. But the result is really far from the ideal case.

### However it's easy to notice that the applied algotrithm has recognized digits ``0, 1, 2, 6`` pretty well.

In [44]:
print(len(Kset[2].difference(Rset[2])))

729


In [70]:
draw_image(df.iloc[kmean_batch[9][1], 0], R, C)

In [271]:
s1 = set(np.argwhere(real_labs  == fst1).flatten());
s2 = set(np.argwhere(kmean_labs == 9).flatten());
print(len(s1), len(s2), len(s1.intersection(s2)));

5949 6530 69
